In [ ]:
import sys; sys.path.append('../..')

from src.results import read_results

In [ ]:
strat_names = [
    # 'Apples',
    'ApplesOptions',
    # 'chipdip',
    'SPY-from-2020-09-01-to-2022-08-01'
]
print("Orders")
for strat_name in strat_names:
    print(strat_name, len(list(read_results.get_orders(strat_name))))

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100
from src.risk import simulate_account
import plotly.express as px
import pandas as pd

In [ ]:
returns_by_strat = {}
for strat_name in strat_names:
    print(strat_name, 'getting returns...')
    orders = list(read_results.get_orders(strat_name))
    simulation = simulate_account.Simulation.from_orders(orders, simulate_account.IdealAccountState.empty(simulate_account.build_td_simulation()))
    # NOTE: this can be slow if fetching candles.
    # Can make a script to pre-fetch all days for ticker at once for cache
    # (instead of 1 day at a time per ticker)
    returns = simulation.get_returns()
    returns_by_strat[strat_name] = returns
    print(strat_name, "finished.")

# Time-aligned

In [ ]:
px.line(pd.DataFrame({key: (1 + returns).cumprod() - 1 for key, returns in returns_by_strat.items()}))

# Correlation of Returning Seasons

In [ ]:
# px.line(pd.DataFrame({key: returns.rolling(window=22).mean() for key, returns in returns_by_strat.items()}))
px.line(pd.DataFrame({key: returns.ewm(span=22, min_periods=22).mean() for key, returns in returns_by_strat.items()}))

In [ ]:
df_corr = pd.DataFrame(returns_by_strat).corr()
import numpy as np
import seaborn as sns

sns.heatmap(df_corr, annot=True, vmin=-1, vmax=1, center=0, cmap='vlag', mask=np.triu(np.ones_like(df_corr, dtype=bool))).plot()
df_corr

# Equity Curves

In [ ]:
equity_curves = {strat_name: ((1 + returns).cumprod() - 1).set_axis(range(len(returns))) for strat_name, returns in returns_by_strat.items()}
px.line(pd.DataFrame(equity_curves))